## A CNN Architecture for MNIST
conv -> relu -> pool -> conv -> relu -> pool -> fully connected -> softmax

In [1]:
from __future__ import print_function
from __future__ import division
from __future__ import print_function

import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import time 
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

def make_dir(path):
    """ Create a directory if there isn't one already. """
    try:
        os.mkdir(path)
    except OSError:
        pass


make_dir('checkpoints')
make_dir('checkpoints/convnet_mnist')

# Step 1: load MNIST data to data/mnist
mnist = input_data.read_data_sets("./data/mnist", one_hot=True)

# Step 2: Define paramaters for the model
N_CLASSES = 10
LEARNING_RATE = 0.001
BATCH_SIZE = 128
SKIP_STEP = 10
DROPOUT = 0.75
N_EPOCHS = 1

Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz


In [ ]:
# Step 3: create placeholders for features and labels
# - Each image in the MNIST data is of shape 28*28 = 784 therefore, each image is represented with a 1x784 tensor
# - We'll be doing dropout for hidden layer so we'll need a placeholder for the dropout probability too
# - Use None for shape so we can change the batch_size once we've built the graph
with tf.name_scope('data'):
    X = tf.placeholder(tf.float32, [None, 784], name="X_placeholder")
    Y = tf.placeholder(tf.float32, [None, 10], name="Y_placeholder")

dropout = tf.placeholder(tf.float32, name='dropout')

__Note:__
- Use None for shape so we can change the batch_size once we've built the graph
- One shape dimension can be -1. In this case, the value is inferred from the length of the array and remaining dimensions.

In [ ]:
# Step 4 + 5: create weights + do inference
# the model is conv -> relu -> pool -> conv -> relu -> pool -> fully connected -> softmax

global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')

with tf.variable_scope('conv1') as scope:
    # first, reshape the image to [BATCH_SIZE, 28, 28, 1] to make it work with tf.nn.conv2d
    # use the dynamic dimension -1
    
    images = tf.reshape(X, shape=[-1, 28, 28, 1]) 
    
    # TO DO

    # create kernel variable of dimension [5, 5, 1, 32]
    # use tf.truncated_normal_initializer()
    # For conv2d: Given an input tensor of shape [batch, in_height, in_width, in_channels] 
    # and a filter / kernel tensor of shape [filter_height, filter_width, in_channels, out_channels]
    conv1_kernels =  tf.get_variable(shape=[5, 5, 1, 32], initializer=tf.truncated_normal_initializer(mean=0, stddev=1), 
                                     name='conv1kernels', dtype=tf.float32)
    
    # TO DO

    # create biases variable of dimension [32]
    # use tf.constant_initializer(0.0)
    conv1_biases = tf.Variable([32], tf.constant_initializer(0.0), name='conv1biases', dtype=tf.float32)
    
    # TO DO 

    # apply tf.nn.conv2d. strides [1, 1, 1, 1], padding is 'SAME'
    
    conv1_output = tf.nn.conv2d(images, conv1_kernels, strides=[1,1,1,1], padding='SAME', name='conv1')
    # TO DO

    # apply relu on the sum of convolution output and biases
    relu1 = tf.nn.relu(tf.add(conv1_output, conv1_biases), name='relu1')
    
    # TO DO 

    # output is of dimension BATCH_SIZE x 28 x 28 x 32

with tf.variable_scope('pool1') as scope:
    # apply max pool with ksize [1, 2, 2, 1], and strides [1, 2, 2, 1], padding 'SAME'
    # stride of 2,2 means non-overlapping
    max_pool1 = tf.nn.max_pool(relu1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME', name='maxpool1')
    # TO DO

    # output is of dimension BATCH_SIZE x 14 x 14 x 32

with tf.variable_scope('conv2') as scope:
    # similar to conv1, except kernel now is of the size 5 x 5 x 32 x 64
    # 32 is the input channels (comes from previous layers)
    conv2_kernels = tf.get_variable('conv2_kernels', [5, 5, 32, 64], initializer=tf.truncated_normal_initializer())
    conv2_biases = tf.get_variable('conv2_biases', [64], initializer=tf.random_normal_initializer()) # TODO: WHY not 0?
    conv2_output = tf.nn.conv2d(max_pool1, conv2_kernels, strides=[1, 1, 1, 1], padding='SAME', name='conv2output')
    relu2 = tf.nn.relu(conv2_output + conv2_biases, name='relu2')

    # output is of dimension BATCH_SIZE x 14 x 14 x 64

with tf.variable_scope('pool2') as scope:
    # similar to pool1
    max_pool2 = tf.nn.max_pool(relu2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='maxpool2')

    # output is of dimension BATCH_SIZE x 7 x 7 x 64

with tf.variable_scope('fc') as scope:
    # use weight of dimension 7 * 7 * 64 x 1024
    input_features = 7 * 7 * 64
    
    # create weights and biases
    fc_weight = tf.Variable(tf.random_normal([input_features, 1024]), name='fcweight')

    fc_bias = tf.Variable(tf.zeros([1024]), name='fcbias')


    # TO DO

    # reshape pool2 to 2 dimensional
    max_pool2 = tf.reshape(max_pool2, [-1, input_features])

    # apply relu on matmul of pool2 and w + b
    fc = tf.nn.relu(tf.matmul(max_pool2, fc_weight) + fc_bias, name='fcrelu')
    
    # TO DO

    # apply dropout
    fc = tf.nn.dropout(fc, dropout, name='dropout')

with tf.variable_scope('softmaxlinear', reuse=True) as scope:
    # this you should know. get logits without softmax
    # you need to create weights and biases
    
    W = tf.Variable(tf.random_normal(shape=[1024,10], mean=0, stddev=0.1), name="W")
    b = tf.Variable(tf.zeros(shape=[10]), name="b")
    logits = tf.matmul(fc, W) + b

    

    # TO DO

# Step 6: define loss function
# use softmax cross entropy with logits as the loss function
# compute mean cross entropy, softmax is applied internally
with tf.name_scope('loss'):
    # you should know how to do this too
    entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y, name="entropy")
    loss = tf.reduce_mean(entropy)

    # TO DO

# Step 7: define training op
# using gradient descent with learning rate of LEARNING_RATE to minimize cost
# don't forgot to pass in global_step
    optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE)
    lossOptimizer = optimizer.minimize(loss, global_step=global_step)
# TO DO

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    # to visualize using TensorBoard
    writer = tf.summary.FileWriter('./my_graph/mnist', sess.graph)
    ##### You have to create folders to store checkpoints
    ckpt = tf.train.get_checkpoint_state(os.path.dirname('checkpoints/convnet_mnist/checkpoint'))
    # if that checkpoint exists, restore from checkpoint
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
    
    initial_step = global_step.eval()

    start_time = time.time()
    n_batches = int(mnist.train.num_examples / BATCH_SIZE)

    total_loss = 0.0
    for index in range(initial_step, n_batches * N_EPOCHS): # train the model n_epochs times
        X_batch, Y_batch = mnist.train.next_batch(BATCH_SIZE)
        _, loss_batch = sess.run([lossOptimizer, loss], 
                                feed_dict={X: X_batch, Y:Y_batch, dropout: DROPOUT}) 
        total_loss += loss_batch
        if (index + 1) % SKIP_STEP == 0:
            print('Average loss at step {}: {:5.1f}'.format(index + 1, total_loss / SKIP_STEP))
            total_loss = 0.0
            saver.save(sess, 'checkpoints/convnet_mnist/mnist-convnet', index)
    
    print("Optimization Finished!") # should be around 0.35 after 25 epochs
    print("Total time: {0} seconds".format(time.time() - start_time))
    
    # test the model
    n_batches = int(mnist.test.num_examples/BATCH_SIZE)
    total_correct_preds = 0
    for i in range(n_batches):
        X_batch, Y_batch = mnist.test.next_batch(BATCH_SIZE)
        _, loss_batch, logits_batch = sess.run([optimizer, loss, logits], 
                                        feed_dict={X: X_batch, Y:Y_batch, dropout: DROPOUT}) 
        preds = tf.nn.softmax(logits_batch)
        correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y_batch, 1))
        accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))
        total_correct_preds += sess.run(accuracy)   
    
    print("Accuracy {0}".format(total_correct_preds/mnist.test.num_examples))

Average loss at step 10: 71733.0
Average loss at step 20: 46328.8
Average loss at step 30: 36165.9
Average loss at step 40: 28567.6
Average loss at step 50: 22187.0
Average loss at step 60: 17887.5
Average loss at step 70: 13827.2
